# Project 1 

## Group One: Covid 19 | Part One: Data Cleansing 

Team Members: Michael Bien, Lupita Lopez, Jinah Porter, Debra Wu

### Research Questions to Answer:

1. WORLD: Compare COVID deaths by GDP by Country: Does a higher GDP result in a lower number of COVID deaths?

2. US: Examine death rates by age group and determine is children under 10 die at a lower rate, adults 60+ at a higher rate

3. US: Compare cases/deaths by state

4. US: Does pre-COVID state GDP related to the cases/deaths in a given state

5. March shutdown, 4th of July and Memorial Day: compare 2 months of shutdown vs. reopening: how did reopening affect case and death rates? 

6. US: Is there a relationship between COVID cases/deaths and gender?

7. US: Is there a relationship between COVID cases/deaths and ethnicity?


In [2]:
#import
import pandas as pd
import os
import requests
import numpy as np
import requests
from pprint import pprint

In [3]:
url = 'https://api.covidtracking.com/v1/states/current.json'

print(url)


https://api.covidtracking.com/v1/states/current.json


In [4]:
print(requests.get(url))

<Response [200]>


In [5]:
response = requests.get(url).json()

pprint(response)



[{'checkTimeEt': '09/15 23:59',
  'commercialScore': 0,
  'dataQualityGrade': 'A',
  'date': 20200916,
  'dateChecked': '2020-09-16T03:59:00Z',
  'dateModified': '2020-09-16T03:59:00Z',
  'death': 44,
  'deathConfirmed': 44,
  'deathIncrease': 0,
  'deathProbable': None,
  'fips': '02',
  'grade': '',
  'hash': 'ad1f0e91a77746e4f6d9ba6cc959464870a19357',
  'hospitalized': None,
  'hospitalizedCumulative': None,
  'hospitalizedCurrently': 45,
  'hospitalizedIncrease': 0,
  'inIcuCumulative': None,
  'inIcuCurrently': None,
  'lastUpdateEt': '9/16/2020 03:59',
  'negative': 401199,
  'negativeIncrease': 1388,
  'negativeRegularScore': 0,
  'negativeScore': 0,
  'negativeTestsAntibody': None,
  'negativeTestsPeopleAntibody': None,
  'negativeTestsViral': 401615,
  'onVentilatorCumulative': None,
  'onVentilatorCurrently': 9,
  'pending': None,
  'posNeg': 408564,
  'positive': 7365,
  'positiveCasesViral': 7365,
  'positiveIncrease': 52,
  'positiveScore': 0,
  'positiveTestsAntibody': No

  'onVentilatorCumulative': None,
  'onVentilatorCurrently': None,
  'pending': None,
  'posNeg': 1394405,
  'positive': 55166,
  'positiveCasesViral': 52878,
  'positiveIncrease': 135,
  'positiveScore': 0,
  'positiveTestsAntibody': None,
  'positiveTestsAntigen': None,
  'positiveTestsPeopleAntibody': None,
  'positiveTestsPeopleAntigen': None,
  'positiveTestsViral': None,
  'recovered': 9142,
  'score': 0,
  'state': 'CT',
  'total': 1394405,
  'totalTestEncountersViral': None,
  'totalTestResults': 1394405,
  'totalTestResultsIncrease': 3634,
  'totalTestResultsSource': 'posNeg',
  'totalTestsAntibody': None,
  'totalTestsAntigen': None,
  'totalTestsPeopleAntibody': None,
  'totalTestsPeopleAntigen': None,
  'totalTestsPeopleViral': None,
  'totalTestsViral': 1392117},
 {'checkTimeEt': '09/14 20:00',
  'commercialScore': 0,
  'dataQualityGrade': 'A+',
  'date': 20200916,
  'dateChecked': '2020-09-15T00:00:00Z',
  'dateModified': '2020-09-15T00:00:00Z',
  'death': 617,
  'deathCo

  'totalTestsPeopleAntigen': None,
  'totalTestsPeopleViral': 1261892,
  'totalTestsViral': 1770032},
 {'checkTimeEt': '09/16 06:00',
  'commercialScore': 0,
  'dataQualityGrade': 'A+',
  'date': 20200916,
  'dateChecked': '2020-09-16T10:00:00Z',
  'dateModified': '2020-09-16T10:00:00Z',
  'death': 586,
  'deathConfirmed': None,
  'deathIncrease': 52,
  'deathProbable': None,
  'fips': '20',
  'grade': '',
  'hash': '51ad37f734d7195edebb25bca14e749bf6192912',
  'hospitalized': 2616,
  'hospitalizedCumulative': 2616,
  'hospitalizedCurrently': 278,
  'hospitalizedIncrease': 44,
  'inIcuCumulative': 713,
  'inIcuCurrently': 83,
  'lastUpdateEt': '9/16/2020 10:00',
  'negative': 413356,
  'negativeIncrease': 4874,
  'negativeRegularScore': 0,
  'negativeScore': 0,
  'negativeTestsAntibody': None,
  'negativeTestsPeopleAntibody': None,
  'negativeTestsViral': None,
  'onVentilatorCumulative': 225,
  'onVentilatorCurrently': 21,
  'pending': None,
  'posNeg': 464226,
  'positive': 50870,
  

  'fips': '31',
  'grade': '',
  'hash': 'ce01765e4f9231ad52e9f0c82b0bc6289d712f4c',
  'hospitalized': 2167,
  'hospitalizedCumulative': 2167,
  'hospitalizedCurrently': 167,
  'hospitalizedIncrease': 13,
  'inIcuCumulative': None,
  'inIcuCurrently': None,
  'lastUpdateEt': '9/15/2020 20:05',
  'negative': 369991,
  'negativeIncrease': 1958,
  'negativeRegularScore': 0,
  'negativeScore': 0,
  'negativeTestsAntibody': None,
  'negativeTestsPeopleAntibody': None,
  'negativeTestsViral': None,
  'onVentilatorCumulative': None,
  'onVentilatorCurrently': None,
  'pending': None,
  'posNeg': 408961,
  'positive': 38970,
  'positiveCasesViral': 38970,
  'positiveIncrease': 328,
  'positiveScore': 0,
  'positiveTestsAntibody': None,
  'positiveTestsAntigen': None,
  'positiveTestsPeopleAntibody': None,
  'positiveTestsPeopleAntigen': None,
  'positiveTestsViral': None,
  'recovered': 29799,
  'score': 0,
  'state': 'NE',
  'total': 408961,
  'totalTestEncountersViral': None,
  'totalTestRes

  'posNeg': 5299620,
  'positive': 674772,
  'positiveCasesViral': 674772,
  'positiveIncrease': 6026,
  'positiveScore': 0,
  'positiveTestsAntibody': 33463,
  'positiveTestsAntigen': 7131,
  'positiveTestsPeopleAntibody': None,
  'positiveTestsPeopleAntigen': None,
  'positiveTestsViral': None,
  'recovered': 590837,
  'score': 0,
  'state': 'TX',
  'total': 5299620,
  'totalTestEncountersViral': None,
  'totalTestResults': 5299620,
  'totalTestResultsIncrease': 52078,
  'totalTestResultsSource': 'posNeg',
  'totalTestsAntibody': 377019,
  'totalTestsAntigen': 52679,
  'totalTestsPeopleAntibody': None,
  'totalTestsPeopleAntigen': None,
  'totalTestsPeopleViral': None,
  'totalTestsViral': 5299620},
 {'checkTimeEt': '09/16 06:38',
  'commercialScore': 0,
  'dataQualityGrade': 'A+',
  'date': 20200916,
  'dateChecked': '2020-09-16T10:38:00Z',
  'dateModified': '2020-09-16T10:38:00Z',
  'death': 437,
  'deathConfirmed': None,
  'deathIncrease': 1,
  'deathProbable': None,
  'fips': '49

In [6]:
len(response)
state = []
num_state = range(0,56)
date = []
death = []


for i in num_state:
    (print(response[i]['state']))
    state.append(response[i]['state'])
    (print(response[i]['date']))
    date.append(response[i]['date'])
    (print(response[i]['deathConfirmed']))
    death.append(response[i]['death'])


AK
20200916
44
AL
20200916
2257
AR
20200916
1010
AS
20200916
None
AZ
20200916
5098
CA
20200916
None
CO
20200916
None
CT
20200916
3592
DC
20200916
None
DE
20200916
547
FL
20200916
13100
GA
20200916
None
GU
20200916
None
HI
20200916
None
IA
20200916
None
ID
20200916
385
IL
20200916
8367
IN
20200916
3247
KS
20200916
None
KY
20200916
1073
LA
20200916
5126
MA
20200916
9036
MD
20200916
3712
ME
20200916
137
MI
20200916
6623
MN
20200916
1933
MO
20200916
None
MP
20200916
2
MS
20200916
2562
MT
20200916
None
NC
20200916
3149
ND
20200916
138
NE
20200916
None
NH
20200916
None
NJ
20200916
14263
NM
20200916
None
NV
20200916
None
NY
20200916
None
OH
20200916
4256
OK
20200916
None
OR
20200916
None
PA
20200916
None
PR
20200916
403
RI
20200916
None
SC
20200916
2968
SD
20200916
None
TN
20200916
2074
TX
20200916
None
UT
20200916
None
VA
20200916
2711
VI
20200916
19
VT
20200916
58
WA
20200916
2020
WI
20200916
1228
WV
20200916
288
WY
20200916
None


In [7]:
clean_df = pd.DataFrame({'State': state,
                         'Date': date,
                        'Number of Deaths': death})

clean_df

states_df = clean_df.drop(clean_df.index[[3, 12, 27, 42, 50]])

new = states_df.reset_index().drop(columns='index')

ok = new.replace({'AK': 'Alaska', 'AL':'Alabama', 'AR': 'Arkansas', 'AZ':'Arizona',
                 'CA': 'California', 'CO': 'Colorado','CT': 'Connecticut',
                 'DC': 'District of Columbia', 'DE': 'Delaware',
                 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii',
                 'IA': 'Iowa', 'ID': 'Idaho', 'IL': 'Illinois',
                 'IN': 'Indiana', 'KS': 'Kansas', 'KY': 'Kentucky',
                 'LA': 'Louisiana', 'MA': 'Massachusetts', 'MD': 'Maryland',
                 'ME': 'Maine', 'MI': 'Michigan', 'MN': 'Minnesota',
                 'MO': 'Missouri', 'MS': 'Mississippi', 'MT': 'Montana',
                 'NC': 'North Carolina', 'ND': 'North Dakota', 'NE': 'Nebraska',
                 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico',
                 'NV': 'Nevada', 'NY': 'New York', 'OH': 'Ohio',
                 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania',
                 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota',
                 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah',
                 'VA': 'Virginia', 'VT': 'Vermont', 'WA': 'Washington',
                 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'})

ok

,State,Date,Number of Deaths
0,Alaska,20200916,44
1,Alabama,20200916,2392
2,Arkansas,20200916,1157
3,Arizona,20200916,5371
4,California,20200916,14615
5,Colorado,20200916,1905
6,Connecticut,20200916,4487
7,District of Columbia,20200916,617
8,Delaware,20200916,619
9,Florida,20200916,13100


In [8]:
 
# quotechar='"'
# , sep=';'
gdp_df.head(25)




,GeoFips,GeoName,LineCode,Description,2018:Q1,2018:Q2,2018:Q3,2018:Q4
0,0,United States *,1.0,All industry total,20163159,20510177,20749752,20897804
1,0,United States *,2.0,Private industries,17654891,17977475,18189960,18320015
2,0,United States *,3.0,"Agriculture, forestry, fishing and hunting",169446,172100,160510,163799
3,0,United States *,6.0,"Mining, quarrying, and oil and gas extra...",328770,346432,361288,350091
4,0,United States *,10.0,Utilities,322592,326273,323103,331440
5,0,United States *,11.0,Construction,824254,835302,845371,851643
6,0,United States *,12.0,Manufacturing,2274772,2308125,2337156,2364714
7,0,United States *,13.0,Durable goods manufacturing,1272352,1286168,1305727,1321374
8,0,United States *,25.0,Nondurable goods manufacturing,1002420,1021956,1031428,1043341
9,0,United States *,34.0,Wholesale trade,1184398,1200091,1217788,1246640


In [9]:
gdp_df.isnull().any()

gdp_df.count()

gdp_df[gdp_df.isnull().any(axis=1)]

clean_df = gdp_df.dropna(how='any')

clean_df.count()

clean_df.head()

,GeoFips,GeoName,LineCode,Description,2018:Q1,2018:Q2,2018:Q3,2018:Q4
0,0,United States *,1.0,All industry total,20163159,20510177,20749752,20897804
1,0,United States *,2.0,Private industries,17654891,17977475,18189960,18320015
2,0,United States *,3.0,"Agriculture, forestry, fishing and hunting",169446,172100,160510,163799
3,0,United States *,6.0,"Mining, quarrying, and oil and gas extra...",328770,346432,361288,350091
4,0,United States *,10.0,Utilities,322592,326273,323103,331440


In [10]:
#drop geofips & line code, not sure we need them
new_df = clean_df[['GeoName', 'Description', '2018:Q1', '2018:Q2', '2018:Q3', '2018:Q4']]

In [11]:
new_df

,GeoName,Description,2018:Q1,2018:Q2,2018:Q3,2018:Q4
0,United States *,All industry total,20163159,20510177,20749752,20897804
1,United States *,Private industries,17654891,17977475,18189960,18320015
2,United States *,"Agriculture, forestry, fishing and hunting",169446,172100,160510,163799
3,United States *,"Mining, quarrying, and oil and gas extra...",328770,346432,361288,350091
4,United States *,Utilities,322592,326273,323103,331440
...,...,...,...,...,...,...
1438,Far West,Health care and social assistance,258781.8,262129,265730.2,268916.8
1439,Far West,"Arts, entertainment, and recreation",52251.6,55032.1,55224.1,56127.1
1440,Far West,Accommodation and food services,143296.6,145704.5,147121.8,150429
1441,Far West,Other services (except government and go...,77503.7,78638.8,78846.7,80519.2


In [12]:
# quarter_df = new_df[['2018:Q1', '2018:Q2', '2018:Q3', '2018:Q4']]

values_df = new_df[new_df['2018:Q1'] != '(D)'][['2018:Q1', '2018:Q2', '2018:Q3', '2018:Q4']].astype(float)

merge_df = new_df.merge(values_df, left_index=True, right_index=True)

merge_df.dtypes

#remove columns 
first_org = merge_df.drop(columns = ['2018:Q1_x', '2018:Q2_x', '2018:Q3_x', '2018:Q4_x'])

#rename columns 
new_df = first_org.rename(columns={"2018:Q1_y": "2018:Q1", "2018:Q2_y": "2018:Q2","2018:Q3_y": "2018:Q3", "2018:Q4_y": "2018:Q4"})

new_df

,GeoName,Description,2018:Q1,2018:Q2,2018:Q3,2018:Q4
0,United States *,All industry total,20163159.0,20510177.0,20749752.0,20897804.0
1,United States *,Private industries,17654891.0,17977475.0,18189960.0,18320015.0
2,United States *,"Agriculture, forestry, fishing and hunting",169446.0,172100.0,160510.0,163799.0
3,United States *,"Mining, quarrying, and oil and gas extra...",328770.0,346432.0,361288.0,350091.0
4,United States *,Utilities,322592.0,326273.0,323103.0,331440.0
...,...,...,...,...,...,...
1438,Far West,Health care and social assistance,258781.8,262129.0,265730.2,268916.8
1439,Far West,"Arts, entertainment, and recreation",52251.6,55032.1,55224.1,56127.1
1440,Far West,Accommodation and food services,143296.6,145704.5,147121.8,150429.0
1441,Far West,Other services (except government and go...,77503.7,78638.8,78846.7,80519.2


In [13]:
new_df['Total GDP'] = new_df.sum(axis = 1)

new_df

,GeoName,Description,2018:Q1,2018:Q2,2018:Q3,2018:Q4,Total GDP
0,United States *,All industry total,20163159.0,20510177.0,20749752.0,20897804.0,82320892.0
1,United States *,Private industries,17654891.0,17977475.0,18189960.0,18320015.0,72142341.0
2,United States *,"Agriculture, forestry, fishing and hunting",169446.0,172100.0,160510.0,163799.0,665855.0
3,United States *,"Mining, quarrying, and oil and gas extra...",328770.0,346432.0,361288.0,350091.0,1386581.0
4,United States *,Utilities,322592.0,326273.0,323103.0,331440.0,1303408.0
...,...,...,...,...,...,...,...
1438,Far West,Health care and social assistance,258781.8,262129.0,265730.2,268916.8,1055557.8
1439,Far West,"Arts, entertainment, and recreation",52251.6,55032.1,55224.1,56127.1,218634.9
1440,Far West,Accommodation and food services,143296.6,145704.5,147121.8,150429.0,586551.9
1441,Far West,Other services (except government and go...,77503.7,78638.8,78846.7,80519.2,315508.4


In [14]:
new_df.dtypes



GeoName         object
Description     object
2018:Q1        float64
2018:Q2        float64
2018:Q3        float64
2018:Q4        float64
Total GDP      float64
dtype: object

In [15]:
new_df.head()

## change to int and then add 

,GeoName,Description,2018:Q1,2018:Q2,2018:Q3,2018:Q4,Total GDP
0,United States *,All industry total,20163159.0,20510177.0,20749752.0,20897804.0,82320892.0
1,United States *,Private industries,17654891.0,17977475.0,18189960.0,18320015.0,72142341.0
2,United States *,"Agriculture, forestry, fishing and hunting",169446.0,172100.0,160510.0,163799.0,665855.0
3,United States *,"Mining, quarrying, and oil and gas extra...",328770.0,346432.0,361288.0,350091.0,1386581.0
4,United States *,Utilities,322592.0,326273.0,323103.0,331440.0,1303408.0


In [16]:
all_industries = new_df[new_df['Description'] == 'All industry total']

In [17]:
all_industries.reset_index()

,index,GeoName,Description,2018:Q1,2018:Q2,2018:Q3,2018:Q4,Total GDP
0,0,United States *,All industry total,20163159.0,20510177.0,20749752.0,20897804.0,82320892.0
1,27,Alabama,All industry total,217369.7,220572.2,223140.7,225859.4,886942.0
2,51,Alaska,All industry total,53563.0,54372.7,55410.0,55590.6,218936.3
3,75,Arizona,All industry total,340672.6,345474.0,351708.8,355332.9,1393188.3
4,99,Arkansas,All industry total,126088.8,128352.5,129032.8,130201.5,513675.6
5,123,California,All industry total,2928096.5,2992622.8,3017340.5,3052871.3,11990931.1
6,147,Colorado,All industry total,362721.2,370476.6,375311.6,378488.8,1486998.2
7,171,Connecticut,All industry total,272854.5,273884.5,278294.5,277874.1,1102907.6
8,195,Delaware,All industry total,72091.2,73024.7,74461.4,74348.1,293925.4
9,219,District of Columbia,All industry total,138469.2,139579.6,141968.8,142626.2,562643.8


In [18]:
ind_df = all_industries.drop(all_industries.index[[0, 52, 53, 54, 55, 56, 57, 58, 59]]).reset_index()

clean_ind = ind_df.reset_index()

ind_df_clean = clean_ind.drop(columns = ['index', 'level_0'])

state_gdp = ind_df_clean.rename(columns={'GeoName': 'State'})


In [19]:
state_gdp

,State,Description,2018:Q1,2018:Q2,2018:Q3,2018:Q4,Total GDP
0,Alabama,All industry total,217369.7,220572.2,223140.7,225859.4,886942.0
1,Alaska,All industry total,53563.0,54372.7,55410.0,55590.6,218936.3
2,Arizona,All industry total,340672.6,345474.0,351708.8,355332.9,1393188.3
3,Arkansas,All industry total,126088.8,128352.5,129032.8,130201.5,513675.6
4,California,All industry total,2928096.5,2992622.8,3017340.5,3052871.3,11990931.1
5,Colorado,All industry total,362721.2,370476.6,375311.6,378488.8,1486998.2
6,Connecticut,All industry total,272854.5,273884.5,278294.5,277874.1,1102907.6
7,Delaware,All industry total,72091.2,73024.7,74461.4,74348.1,293925.4
8,District of Columbia,All industry total,138469.2,139579.6,141968.8,142626.2,562643.8
9,Florida,All industry total,1020176.9,1032277.4,1048477.6,1056013.7,4156945.6


In [20]:
state_gdp = state_gdp.merge(ok, on='State', how='left')

In [ ]:
state_gdp

In [27]:
#get the data from the CSV file for population

f_path = os.path.join('Resources', 'csvData.csv')

pop_df = pd.read_csv(f_path)

pop_df.head()

#only use 2018 data 

state_pop = pop_df[['State', 'Pop2018']]

In [122]:
total_df = state_gdp.merge(state_pop, on='State', how='left')

In [123]:
total_df.dtypes

State                object
Description          object
2018:Q1             float64
2018:Q2             float64
2018:Q3             float64
2018:Q4             float64
Total GDP           float64
Date                  int64
Number of Deaths      int64
Pop2018               int64
dtype: object

In [124]:
t2 = total_df.drop(columns='Description')

In [125]:
t2

,State,2018:Q1,2018:Q2,2018:Q3,2018:Q4,Total GDP,Date,Number of Deaths,Pop2018
0,Alabama,217369.7,220572.2,223140.7,225859.4,886942.0,20200916,2392,4887870
1,Alaska,53563.0,54372.7,55410.0,55590.6,218936.3,20200916,44,737438
2,Arizona,340672.6,345474.0,351708.8,355332.9,1393188.3,20200916,5371,7171650
3,Arkansas,126088.8,128352.5,129032.8,130201.5,513675.6,20200916,1157,3013820
4,California,2928096.5,2992622.8,3017340.5,3052871.3,11990931.1,20200916,14615,39557000
5,Colorado,362721.2,370476.6,375311.6,378488.8,1486998.2,20200916,1905,5695560
6,Connecticut,272854.5,273884.5,278294.5,277874.1,1102907.6,20200916,4487,3572660
7,Delaware,72091.2,73024.7,74461.4,74348.1,293925.4,20200916,619,967171
8,District of Columbia,138469.2,139579.6,141968.8,142626.2,562643.8,20200916,617,702455
9,Florida,1020176.9,1032277.4,1048477.6,1056013.7,4156945.6,20200916,13100,21299300


In [126]:
newEngland = []
middleAtlantic = []
eastnorthCentral = []

farWest = []
greatLakes = []
midSouth = []
midWest = []
mountainWest = []

northEast = []
northWest = []
southCentral = []
southEast = []

states = t2['State']

for state in states:
    if state == 'Maine' or state == 'New Hampshire' or state == 'Vermont' or state == 'Massachusetts' or state == 'Rhode Island' or state == 'Connecticut':
        newEngland.append(state)
        
    if state == 'New York' or state == 'Pennsylvania' or state == 'New Jersey':
        middleAtlantic.append(state)
        
    if state == 'Wisconsin' or state == 'Michigan' or state == 'Illinois' or state == 'Indiana' or state == 'Ohio':
        eastnorthCentral.append(state)
        
    if state == 'California' or state == 'Hawaii' or state == 'Nevada': 
        farWest.append(state)

    if state == 'Delaware' or state == 'District of Columbia' or state == 'Kentucky' or state == 'Maryland' or state == 'North Carolina' or state == 'Tennessee' or state == 'Virginia' or state == 'West Virginia':
        midSouth.append(state)
    if state == 'Iowa' or state == 'Kansas' or state == 'Missouri' or state == 'Nebraska' or state == 'North Dakota' or state == 'South Dakota':
        midWest.append(state)
    if state == 'Arizona' or state == 'Colorado' or state == 'Idaho' or state == 'Montana' or state == 'New Mexico' or state == 'Utah' or state == 'Wyoming':
        mountainWest.append(state)


    if state == 'Oregon' or state == 'Washington':
        northWest.append(state)
    if state == 'Arkansas' or state == 'Louisiana' or state == 'Oklahoma' or state == 'Texas':
        southCentral.append(state)
    if state == 'Alabama' or state == 'Florida' or state == 'Georgia' or state == 'Mississippi' or state == 'South Carolina':
        southEast.append(state)


In [127]:
newengland = pd.DataFrame({'State': newEngland})
newengland['Region'] = 'New England'

middleatlantic = pd.DataFrame({'State': middleAtlantic})
middleatlantic['Region'] = 'Middle Atlantic'

eastnorthcentral = pd.DataFrame({'State': eastnorthCentral})
eastnorthcentral['Region'] = 'East North Central'

midsouth = pd.DataFrame({'State': midSouth})
midsouth['Region'] = 'Mid South'

midwest = pd.DataFrame({'State': midWest})
midwest['Region'] = 'Mid West'

mountainwest = pd.DataFrame({'State': mountainWest})
mountainwest['Region'] = 'Mountain West'



northeast = pd.DataFrame({'State': northEast})
northeast['Region'] = 'North East'

northwest = pd.DataFrame({'State': northWest})
northwest['Region'] = 'North West'

southcentral = pd.DataFrame({'State': southCentral})
southcentral['Region'] = 'South Central'

southeast = pd.DataFrame({'State': southEast})
southeast['Region'] = 'South East'

frames = [farwest, greatlakes, midsouth, midwest, mountainwest, newengland,
         northeast, northwest, southcentral, southeast]

result = pd.concat(frames).reset_index().drop(columns='index')

In [128]:
result

,State,Region
0,California,Far West
1,Hawaii,Far West
2,Nevada,Far West
3,Illinois,Great Lakes
4,Indiana,Great Lakes
5,Michigan,Great Lakes
6,Minnesota,Great Lakes
7,Ohio,Great Lakes
8,Wisconsin,Great Lakes
9,Delaware,Mid South


In [129]:
complete_df = t2.merge(result, on='State', how='left')

In [131]:
complete_df

,State,2018:Q1,2018:Q2,2018:Q3,2018:Q4,Total GDP,Date,Number of Deaths,Pop2018,Region
0,Alabama,217369.7,220572.2,223140.7,225859.4,886942.0,20200916,2392,4887870,South East
1,Alaska,53563.0,54372.7,55410.0,55590.6,218936.3,20200916,44,737438,NaN
2,Arizona,340672.6,345474.0,351708.8,355332.9,1393188.3,20200916,5371,7171650,Mountain West
3,Arkansas,126088.8,128352.5,129032.8,130201.5,513675.6,20200916,1157,3013820,South Central
4,California,2928096.5,2992622.8,3017340.5,3052871.3,11990931.1,20200916,14615,39557000,Far West
5,Colorado,362721.2,370476.6,375311.6,378488.8,1486998.2,20200916,1905,5695560,Mountain West
6,Connecticut,272854.5,273884.5,278294.5,277874.1,1102907.6,20200916,4487,3572660,New England
7,Delaware,72091.2,73024.7,74461.4,74348.1,293925.4,20200916,619,967171,Mid South
8,District of Columbia,138469.2,139579.6,141968.8,142626.2,562643.8,20200916,617,702455,Mid South
9,Florida,1020176.9,1032277.4,1048477.6,1056013.7,4156945.6,20200916,13100,21299300,South East
